In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
USE ROLE ACCOUNTADMIN;

CREATE DATABASE IF NOT EXISTS ski_gear_support_db;
CREATE SCHEMA IF NOT EXISTS ski_gear_support_schema;

USE ski_gear_support_db.ski_gear_support_schema;

In [ ]:
CREATE or REPLACE file format csvformat
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  type = 'CSV';

CREATE or REPLACE stage call_transcripts_data_stage
  file_format = csvformat
  url = 's3://sfquickstarts/misc/call_transcripts/';

CREATE or REPLACE table CALL_TRANSCRIPTS ( 
  date_created date,
  language varchar(60),
  country varchar(60),
  product varchar(60),
  category varchar(60),
  damage_type varchar(90),
  transcript varchar
);

COPY into CALL_TRANSCRIPTS
  from @call_transcripts_data_stage;

In [ ]:
SELECT transcript from CALL_TRANSCRIPTS

In [ ]:
transcripts = session.sql("SELECT transcript from CALL_TRANSCRIPTS").collect()

In [ ]:
transcripts

In [ ]:
for transcript in transcripts:
    print(transcript['TRANSCRIPT'] + (11 *" \n"))

In [ ]:
print(transcripts[3]['TRANSCRIPT'])

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION ='ANY_REGION';

In [ ]:
from snowflake.cortex import Complete
prompt = """
    Summarize this transcript in less than 200 words. 
Put the product name, defect and summary in JSON format. 
"""

In [ ]:
summary = []

In [ ]:
for transcript in transcripts:
    summary.append(Complete("mistral-7b", prompt + transcript['TRANSCRIPT']))

In [ ]:
for i, summary_item in enumerate(summary):
    print(f"Summary {i+1}:")
    print(summary_item)  